# Exploration of State Capitals in the USA

## Introduction

In this final project, I will explore popular venues in 50 state capitals. 
I target travel agencies who aim to attract tourists to the US Capitals (assuming no effect of corona virus :)).
The aim is to find popular spots in each state capital. I also aim to both distinguish them in their uniqueness and find similarities among them. 
This will help agencies to guide their customers better. They can offer destinations diverse enough to satisfy their customers,
so that customers can have an ultimate the US experince.

I restricted the cities only to the capitals, because I thought that this would be a nice theme to attract tourists who want to 
do an unconventional the US trip, rather than just visiting popular places only in well-known cities. 
Also, it is limited to downtown areas of the capitals. I aim to capture ordinary life, 
so that I can explore the choices of locals in their respective cities.

In this project, I will cluster capitals according to the most common and popular venues. 
And I will explore diverse capitals in detail to help travel agencies to recommend good places to their customers. 


# Data

I will use two different databases. 

The first one is a github page where I scrape latitudes and longtitudes of capitals. 

The second one is Foursquare data. 
From there, I will use explore function to get venues around capitals.
I will then get information regarding the venues.

Mainly; ratings, number of likes, number of tips, number of photos, price category (ranging between 1 and 4)

I will use these to have an estimate of popularity of venues.

The reason I chose to use this is because these are the informations that I can get from Foursquare with a regular account.

I limit the number of venues for each state to 100. They are located in 1500 meters radius from the center of capital. As you will see below not every city has that many venues. But I decided to keep radius the same for every city in order to have a consistent comparison.

Below I show the data sets and an example for further analysis fo the last week.

In [3]:
import requests
import numpy as np
import pandas as pd

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
!pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

import matplotlib.cm as cm
import matplotlib.colors as colors

!pip install folium
print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 92kB 12.7MB/s eta 0:00:01
Folium installed
Libraries imported.


In [4]:
# The code was removed by Watson Studio for sharing.

My credentails:
CLIENT_ID: Hidden
CLIENT_SECRET:Hidden


In [5]:
#scrapes the page with BeautifulSoup
page = requests.get('https://gist.githubusercontent.com/kiwiholmberg/8433478/raw/e07ca8efd15293f9ab7e8279097c3696b51d5e21/gistfile1.html').text

from bs4 import BeautifulSoup
soup = BeautifulSoup(page, 'lxml')
#print(soup.prettify())

In [6]:
#gets the four columns of data
states = []
capital_names = []
capital_latitudes = []
capital_longitudes = []

for tr in soup.find_all('li'):
        tds = tr.find_all('p')
        if not tds:
            continue
        state,capital_name,capital_latitude,capital_longitude = ([td.text.strip() for td in tds[:4]])
        states.append(state.replace('Name: ',''))
        capital_names.append(capital_name.replace('Capital Name: ',''))
        capital_latitudes.append(float(capital_latitude.replace('Capital Latitude: ','')))
        capital_longitudes.append(float(capital_longitude.replace('Capital Longitude: ','')))

In [7]:
#writes the data to DataFrame
state_info = pd.DataFrame()
state_info['State'] = states
state_info['Capital'] = capital_names
state_info['Latitude'] = capital_latitudes
state_info['Longitude'] = capital_longitudes
state_info

,State,Capital,Latitude,Longitude
0,Alabama,Montgomery,32.361538,-86.279118
1,Alaska,Juneau,58.301935,-134.419740
2,Arizona,Phoenix,33.448457,-112.073844
3,Arkansas,Little Rock,34.736009,-92.331122
4,California,Sacramento,38.555605,-121.468926
5,Colorado,Denver,39.739167,-104.984167
6,Connecticut,Hartford,41.767000,-72.677000
7,Delaware,Dover,39.161921,-75.526755
8,Florida,Tallahassee,30.451800,-84.272770
9,Georgia,Atlanta,33.760000,-84.390000


In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['id'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Capital', 
                  'Capital Latitude', 
                  'Capital Longitude', 
                  'Venue', 
                  'Venue ID',          
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            32.361538, 
            -86.279118, 
            10000, 
            10)
        # make the GET request
results = requests.get(url).json()["response"]['groups'][0]['items']
for v in results:
    print(v['venue']['name'], v['venue']['id'])

Chick-fil-A 4b50c761f964a520133227e3
Leroy 4f3c79e2e4b07254f705e1aa
Shashy's Bakery & Fine Foods 4dcabad652b1c2222a89cc50
Sushi Cafe 4dac89988154abafc2d9e305
Publix 4b9bc70bf964a520ee2336e3
First Watch - Montgomery 576b2c85cd10f350e1b9006b
Midtown Pizza Kitchen 4da855c293a021ab13c6d644
Filet & Vine 4b9686a9f964a52034d234e3
La Zona Rosa 4bad3674f964a520d3393be3
Tomatinos Pizza and Bake Shop 4b6fc006f964a52086fb2ce3


In [27]:
capital_venues = getNearbyVenues(names=state_info['Capital'],
                                   latitudes=state_info['Latitude'],
                                   longitudes=state_info['Longitude']
                                  )

Montgomery
Juneau
Phoenix
Little Rock
Sacramento
Denver
Hartford
Dover
Tallahassee
Atlanta
Honolulu
Boise
Springfield
Indianapolis
Des Moines
Topeka
Frankfort
Baton Rouge
Augusta
Annapolis
Boston
Lansing
Saint Paul
Jackson
Jefferson City
Helana
Lincoln
Carson City
Concord
Trenton
Santa Fe
Albany
Raleigh
Bismarck
Columbus
Oklahoma City
Salem
Harrisburg
Providence
Columbia
Pierre
Nashville
Austin
Salt Lake City
Montpelier
Richmond
Olympia
Charleston
Madison
Cheyenne


In [28]:
print(capital_venues.shape)
capital_venues.head()

(3519, 8)


,Capital,Capital Latitude,Capital Longitude,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category
0,Montgomery,32.361538,-86.279118,Shashy's Bakery & Fine Foods,4dcabad652b1c2222a89cc50,32.362289,-86.283226,Bakery
1,Montgomery,32.361538,-86.279118,Martin's Restaurant,4b9fc51af964a520ff3c37e3,32.357262,-86.282862,Fried Chicken Joint
2,Montgomery,32.361538,-86.279118,Chick-fil-A,4b50c761f964a520133227e3,32.368860,-86.270454,Fast Food Restaurant
3,Montgomery,32.361538,-86.279118,Zaxby's Chicken Fingers & Buffalo Wings,4bb234ec35f0c9b6b3f4ba83,32.364406,-86.268742,Fried Chicken Joint
4,Montgomery,32.361538,-86.279118,Subway,4ba2d789f964a520721d38e3,32.357502,-86.283664,Sandwich Place


In [29]:
capital_venues.groupby('Capital').count()

,Capital Latitude,Capital Longitude,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category
Capital,,,,,,,
Albany,100,100,100,100,100,100,100
Annapolis,79,79,79,79,79,79,79
Atlanta,100,100,100,100,100,100,100
Augusta,26,26,26,26,26,26,26
Austin,100,100,100,100,100,100,100
Baton Rouge,15,15,15,15,15,15,15
Boise,75,75,75,75,75,75,75
Boston,61,61,61,61,61,61,61
Carson City,49,49,49,49,49,49,49


In [68]:
carson = capital_venues[capital_venues['Capital'] == 'Carson City']
carson.head()

,Capital,Capital Latitude,Capital Longitude,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category
1767,Carson City,39.160949,-119.753877,Paul Schat's Bakery,4b92881cf964a520970134e3,39.156239,-119.765484,Bakery
1768,Carson City,39.160949,-119.753877,Carson City Aquatic Facility,4bfaf977bbb7c92873830743,39.169057,-119.759542,Gym Pool
1769,Carson City,39.160949,-119.753877,Dutch Bros. Coffee,4c6d42d96af58cfacae58817,39.156386,-119.766733,Coffee Shop
1770,Carson City,39.160949,-119.753877,Sportsman's Warehouse,5339acbd498e75abcda555c8,39.154992,-119.765967,Outdoor Supply Store
1771,Carson City,39.160949,-119.753877,Comma Coffee,4b844baff964a5200a2d31e3,39.162055,-119.767024,Coffee Shop


In [31]:
venues_map = folium.Map(location=[39.160949, -119.753877], zoom_start=14.4) # generate map centred around Carson City


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(carson['Venue Latitude'], carson['Venue Longitude'] ,carson['Venue Category']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

# Below, I present the function that I will use to get details of venues

In [69]:
def get_details(venues_id):  
    
    venue_details = []
    
    for venue_id in venues_id:

        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

        result = requests.get(url).json()
        print(venue_id)
        print(result['response']['venue']['rating'])
        details = [venue_id]
        
        if np.isnan(result['response']['venue']['rating']) == False : 
            details.append(result['response']['venue']['rating'])
        else:
            details.append('N/A')
        if np.isnan(result['response']['venue']['likes']['count']) == False: 
            details.append(result['response']['venue']['likes']['count'])
        else:
            details.append('N/A')
        if np.isnan(result['response']['venue']['tips']['count']) == False: 
            details.append(result['response']['venue']['tips']['count'])
        else:
            details.append('N/A')
        if np.isnan(result['response']['venue']['photos']['count']) == False: 
            details.append(result['response']['venue']['photos']['count'])
        else:
            details.append('N/A')       
        if np.isnan(result['response']['venue']['price']['tier']) == False: 
            details.append(result['response']['venue']['price']['tier'])
        else:
            details.append('N/A')
        
        venue_details.append(details)
        
    venues = pd.DataFrame([item for item in venue_details])
    venues.columns = ['Venue ID',  
                  'Rating', 
                  '# of Likes', 
                  '# of Tips',          
                  '# of Photos', 
                  'Price Category']
    return(venues)

In [ ]:
venues_in_carson = get_details(carson['Venue ID'])

Not complete since I reach my daily quota before I submit the assingment

In [59]:
venues_in_carson

,Venue ID,Rating,# of Likes,# of Tips,# of Photos,Price Category
0,4b92881cf964a520970134e3,N/A,N/A,N/A,N/A,N/A
1,4bfaf977bbb7c92873830743,N/A,N/A,N/A,N/A,N/A
2,4c6d42d96af58cfacae58817,N/A,N/A,N/A,N/A,N/A
3,5339acbd498e75abcda555c8,N/A,N/A,N/A,N/A,N/A
4,4b844baff964a5200a2d31e3,N/A,N/A,N/A,N/A,N/A
5,4b648021f964a52082b82ae3,N/A,N/A,N/A,N/A,N/A
6,5924aef98173cb590e4b97bc,N/A,N/A,N/A,N/A,N/A
7,4bce42c0c564ef3b1f07eef0,N/A,N/A,N/A,N/A,N/A
8,4ba912e0f964a5200b093ae3,N/A,N/A,N/A,N/A,N/A
9,4b8a9e92f964a520977532e3,N/A,N/A,N/A,N/A,N/A
